In [0]:
import numpy as np
import pandas as pd
import io
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
os.chdir("/content/drive/")
!ls

'My Drive'


In [0]:
feat = np.load("/content/drive/My Drive/feat.npy", allow_pickle=True)
path = np.load("/content/drive/My Drive/path.npy", allow_pickle=True)
train = pd.read_csv('/content/drive/My Drive/train.csv')
test = pd.read_csv('/content/drive/My Drive/test.csv')

In [0]:
dataset = pd.DataFrame({"path":path,"features":feat})
train2 = pd.merge(train,
        dataset[["path","features"]],
        left_on = ["path"],
        right_on = ["path"],
        how = "left")
train2.loc[train2['features'] == " "]          

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/__init__.py:1069: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x, y, op)


,path,word,features


In [0]:
print(train2[["path","word"]])

                                               path   word
0      8d37d10e7f97ddea2eca9d39a4cf821b4457b041.wav    one
1      9a8f761be3fa0d0a963f5612ba73e68cc0ad11ba.wav    one
2      314cdc39f628bc68d216498b2080bcc7a549a45f.wav    one
3      cc499e63eee4a3bcca48b5b452df04990df83570.wav    one
4      38cdcc4d9432ce4a2fe63e0998dbca91e64b954a.wav    one
...                                             ...    ...
94819  0a906bec4b8cf35a9f1e07bf173197e5d242aebf.wav  house
94820  dc476b9203287605e20b6d0ae3c2a6b02a6960b1.wav  house
94821  5e59953b71372b0d6ddc795576fde1845c6657de.wav  house
94822  45493058505da2a51e407924e06a35163cb45a9d.wav  house
94823  89717a535d5301648bd3ff1e4a3949922a38d22f.wav  house

[94824 rows x 2 columns]


In [0]:
x_train_1 = train2.iloc[:,2:]
x_train_1 = np.array(x_train_1)

y_train_1 = train2.iloc[:,1:2]
y_train_1 = np.array(y_train_1)

In [0]:
np.unique(y_train_1, return_counts=True)

(array(['backward', 'bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five',
        'follow', 'forward', 'four', 'go', 'happy', 'house', 'learn',
        'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right',
        'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up',
        'visual', 'wow', 'yes', 'zero'], dtype=object),
 array([1499, 1807, 1879, 1837, 1908, 3511, 3379, 3607, 1407, 1402, 3328,
        3478, 1851, 1922, 1414, 3389, 1905, 3526, 3536, 3343, 3449, 3491,
        3382, 3592, 1810, 3466, 3461, 3322, 1566, 3456, 3298, 1427, 1917,
        3625, 3634]))

# Feature engineering / Dimension reduction

In [0]:
x_train_1.shape

(94824, 1)

In [0]:
y_train_1.shape

(94824, 1)

In [0]:
x_train_1[0][0].shape

(99, 13)

In [0]:
a = np.concatenate(x_train_1)
a[0].shape

(99, 13)

In [0]:
def features(features, function):
    new = np.copy(features)
    for i in range(len(features)):
        new[i] = function(new[i], axis=0)
    return new

mean_features = features(a, np.mean) 

######
#   [[1,2,3,4],
#   [1,2,3,5],  --> = [(1+1+1)/3, (2+2+2)/3, (3+3+3)/3, (4+5+6)/3] --> [1,2,3,5] 
#   [1,2,3,6]]
######

In [0]:
c = []
for each in mean_features:
    b = each.tolist()
    c.append(b)
c = np.asarray(c)

c.shape

(94824, 13)

In [0]:
c[0]

array([ -8.51916504, -12.39156342,  -8.04028606,  -2.26878309,
        -5.95670509,  -2.26012301,   2.44487143,  -4.60500002,
         2.56349063,   1.04363096,  -6.20483589,   1.81926596,
        -0.96842885])

In [0]:
np.mean(x_train_1[0][0], axis=0)

array([ -8.519165  , -12.391563  ,  -8.040286  ,  -2.268783  ,
        -5.956705  ,  -2.260123  ,   2.4448714 ,  -4.605     ,
         2.5634906 ,   1.043631  ,  -6.204836  ,   1.819266  ,
        -0.96842885], dtype=float32)

In [0]:
np.unique(y_train_1, return_counts = True)

(array(['backward', 'bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five',
        'follow', 'forward', 'four', 'go', 'happy', 'house', 'learn',
        'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right',
        'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up',
        'visual', 'wow', 'yes', 'zero'], dtype=object),
 array([1499, 1807, 1879, 1837, 1908, 3511, 3379, 3607, 1407, 1402, 3328,
        3478, 1851, 1922, 1414, 3389, 1905, 3526, 3536, 3343, 3449, 3491,
        3382, 3592, 1810, 3466, 3461, 3322, 1566, 3456, 3298, 1427, 1917,
        3625, 3634]))

# One-hot encoding

In [0]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = train['word']
values = array(data)
#print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [0]:
y_train1 = onehot_encoded
y_train2 = integer_encoded
print(y_train1[8000:8010])
print(y_train_1[8000:8010])
print(y_train_1[0:10])
print(y_train2[8000:8010])
print(y_train2[0:10])
print(np.unique(y_train_1))

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0.

In [0]:
print (x_train_1.shape)
print (x_train_1[0][0].shape)
#print (c.shape)

(94824, 1)
(99, 13)


# Padding features with zero

In [0]:
from keras.preprocessing.sequence import pad_sequences

empty = []

for each in x_train_1:
  g = pad_sequences(each, maxlen=100, dtype='float32', padding = "post")
  empty.append(g)

empty = np.asarray(empty)
empty.shape

Using TensorFlow backend.


(94824, 1, 100, 13)

In [0]:
x_arr = []
for elke in empty:
  for elkse in elke:
    x_arr.append(elkse)

x_arr = np.asarray(x_arr)
x_arr.shape

(94824, 100, 13)

In [0]:
x_arr[7]

array([[-13.273373  ,  -1.6485854 ,  -3.7370172 , ...,  -5.803972  ,
         12.484084  ,   9.523147  ],
       [-13.227814  ,  -0.54738706,  -4.4502172 , ...,  -7.335784  ,
         14.287482  ,   6.8734455 ],
       [-13.255228  ,  -0.91048557,  -4.0846944 , ...,  -8.168454  ,
          9.982916  ,   1.478483  ],
       ...,
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]], dtype=float32)

In [0]:
x_train_1[7][0]

array([[-13.273373  ,  -1.6485854 ,  -3.7370172 , ...,  -5.803972  ,
         12.484084  ,   9.523147  ],
       [-13.227814  ,  -0.54738706,  -4.4502172 , ...,  -7.335784  ,
         14.287482  ,   6.8734455 ],
       [-13.255228  ,  -0.91048557,  -4.0846944 , ...,  -8.168454  ,
          9.982916  ,   1.478483  ],
       ...,
       [ -1.4170728 ,  30.832207  ,   4.628281  , ..., -10.161711  ,
          0.8648865 ,  -2.342379  ],
       [ -1.3135529 ,  29.043425  ,  -1.190564  , ..., -15.347728  ,
         -7.2374454 ,  -4.6241336 ],
       [ -1.4021184 ,  25.547808  ,  -3.6245093 , ..., -17.73468   ,
         -4.458716  ,  -6.0434103 ]], dtype=float32)

In [0]:
# Feature dimension
feature_dim_1 = 100
# Second dimension of the feature is dim2
feature_dim_2 = 13
channel = 1


In [0]:
X_train1 = x_arr.reshape(x_arr.shape[0], feature_dim_1, feature_dim_2, channel)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train1, y_train1, test_size=0.2, random_state=45)

# Model 

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import optimizers
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint

adam = keras.optimizers.Adam(lr=0.0001,beta_1 = 0.9, beta_2 = 0.999, epsilon=1e-08)
sgd = keras.optimizers.SGD(lr=0.01, decay=0.001, momentum=0.9, nesterov=True)
filepath = '/content/drive/My Drive/model.hdf5' 
es_cb = EarlyStopping(monitor='val_loss', patience=25, verbose=1, mode='auto')
cp_cb = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
epochs = 300
batch_size = 128
num_classes = 35

model = Sequential()
model.add(Conv2D(1028, kernel_size=(4, 4), input_shape=(100, 13, 1)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.35))
model.add(Conv2D(516, kernel_size=(3, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.35))
model.add(Conv2D(256, kernel_size=(3, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.35))
model.add(Conv2D(128, kernel_size=(3, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.35))
model.add(Flatten())
model.add(Dense(128))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.35))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(X_train, y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1, 
          validation_data=(X_test, y_test), 
          shuffle = True,
          callbacks = [cp_cb,
                       es_cb])


Train on 75859 samples, validate on 18965 samples
Epoch 1/300
75859/75859 [==============================] - 183s 2ms/step - loss: 1.8736 - acc: 0.4820 - val_loss: 1.2949 - val_acc: 0.6416

Epoch 00001: val_loss improved from inf to 1.29486, saving model to /content/drive/My Drive/ML/7_best_weights_of_ML challenge.hdf5
Epoch 2/300
75859/75859 [==============================] - 177s 2ms/step - loss: 1.2923 - acc: 0.6354 - val_loss: 1.0476 - val_acc: 0.7029

Epoch 00002: val_loss improved from 1.29486 to 1.04765, saving model to /content/drive/My Drive/ML/7_best_weights_of_ML challenge.hdf5
Epoch 3/300
75859/75859 [==============================] - 177s 2ms/step - loss: 1.0488 - acc: 0.7000 - val_loss: 0.8440 - val_acc: 0.7582

Epoch 00003: val_loss improved from 1.04765 to 0.84395, saving model to /content/drive/My Drive/ML/7_best_weights_of_ML challenge.hdf5
Epoch 4/300
75859/75859 [==============================] - 177s 2ms/step - loss: 0.8919 - acc: 0.7435 - val_loss: 0.7736 - val_ac

KeyboardInterrupt: ignored

# Test features 

In [0]:
test2 = pd.merge(test,
        dataset[["path","features"]],
        left_on = ["path"],
        right_on = ["path"],
        how = "left")
test2.loc[test2['features'] == " "]

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/__init__.py:1069: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x, y, op)


,path,features


In [0]:
print(test2)

                                               path                                           features
0      4985e0c3784b700688c35818ba69f01b4fa3e8da.wav  [[-10.226124, -37.118977, -15.742347, 20.41904...
1      c3815898eb339919ab56249acae83cf566eb622d.wav  [[-9.548587, -24.748652, 0.9056945, -10.070526...
2      32c4865f292674cc904c5af503bc669c2dbd8843.wav  [[-16.266151, -24.703915, -2.5740085, -6.48961...
3      99600d05d1a861ef9771a7bd8eca0d5f444fce7a.wav  [[-13.158611, -15.501357, -1.4940304, -13.2489...
4      57ece62e925c94a7e6c54916caca1237467ad4d8.wav  [[-12.640213, -23.89759, -1.7320107, -8.165893...
...                                             ...                                                ...
11000  21d4a6af950e635bef9d5a4af619ce3cc186206b.wav  [[-8.614212, -48.211033, 22.917099, 7.979172, ...
11001  f8e8f270bbf9c7aeeceb961c5667b622f7baa208.wav  [[-9.978789, -22.874296, -6.1318507, -14.01644...
11002  223a600994f93fcf869b2731a761b2358dc53c23.wav  [[-6.2538586, -33.87

In [0]:
x_test = test2["features"]
print(x_test)

0        [[-10.226124, -37.118977, -15.742347, 20.41904...
1        [[-9.548587, -24.748652, 0.9056945, -10.070526...
2        [[-16.266151, -24.703915, -2.5740085, -6.48961...
3        [[-13.158611, -15.501357, -1.4940304, -13.2489...
4        [[-12.640213, -23.89759, -1.7320107, -8.165893...
                               ...                        
11000    [[-8.614212, -48.211033, 22.917099, 7.979172, ...
11001    [[-9.978789, -22.874296, -6.1318507, -14.01644...
11002    [[-6.2538586, -33.875256, 2.5920956, 18.999304...
11003    [[-11.038775, -26.702341, -2.0712023, 12.88980...
11004    [[-12.711536, -8.897951, -11.146958, -10.19576...
Name: features, Length: 11005, dtype: object


In [0]:
x_test_1 = test2.iloc[:,1:]
x_test_1 = np.array(x_test_1)


In [0]:
print(x_test_1)

[[array([[-10.226124 , -37.118977 , -15.742347 , ..., -10.20496  ,
          3.995985 ,   3.0630302],
       [ -9.272047 , -45.75425  , -17.257053 , ...,  -5.0275555,
          4.2812357,   6.1339855],
       [ -6.402996 , -44.557823 , -26.819525 , ...,   2.633252 ,
          2.2227523,  -1.8913561],
       ...,
       [-10.197387 , -33.277893 ,   8.122305 , ...,  10.723925 ,
          1.8142297,  -3.8644917],
       [-10.118658 , -35.00419  ,  12.04741  , ...,   5.0253925,
         -9.737252 , -17.234812 ],
       [-10.703615 , -34.22347  ,   7.02948  , ...,   3.3475018,
         -8.188433 ,  -8.465663 ]], dtype=float32)]
 [array([[ -9.548587  , -24.748652  ,   0.9056945 , ...,  -8.580786  ,
         -1.1877716 ,   7.7120013 ],
       [ -6.6715455 , -29.529615  ,  12.843338  , ...,  -8.495968  ,
          4.5410557 ,   0.93591136],
       [ -4.9988422 , -43.738503  ,  16.477749  , ...,  -4.1691666 ,
          0.49300647,   9.219536  ],
       ...,
       [-10.403003  , -23.333366  , -

Padding test 

In [0]:
from keras.preprocessing.sequence import pad_sequences

empty = []

for each in x_test_1:
  g = pad_sequences(each, maxlen=100, dtype='float32', padding = "post")
  empty.append(g)

empty = np.asarray(empty)
empty.shape

(11005, 1, 100, 13)

In [0]:
x_arr = []
for elke in empty:
  for elkse in elke:
    x_arr.append(elkse)

x_arr = np.asarray(x_arr)
x_arr.shape

(11005, 100, 13)

In [0]:
x_arr[0]

array([[-10.226124 , -37.118977 , -15.742347 , ..., -10.20496  ,
          3.995985 ,   3.0630302],
       [ -9.272047 , -45.75425  , -17.257053 , ...,  -5.0275555,
          4.2812357,   6.1339855],
       [ -6.402996 , -44.557823 , -26.819525 , ...,   2.633252 ,
          2.2227523,  -1.8913561],
       ...,
       [-10.118658 , -35.00419  ,  12.04741  , ...,   5.0253925,
         -9.737252 , -17.234812 ],
       [-10.703615 , -34.22347  ,   7.02948  , ...,   3.3475018,
         -8.188433 ,  -8.465663 ],
       [  0.       ,   0.       ,   0.       , ...,   0.       ,
          0.       ,   0.       ]], dtype=float32)

In [0]:
x_test1 = x_arr.reshape(x_arr.shape[0], feature_dim_1, feature_dim_2, channel)

model test

# Predictions


In [0]:
from keras.models import load_model
model = load_model("/content/drive/My Drive/model.hdf5") 

In [0]:
pred =  model.predict(x_test1)

In [0]:
pred.shape

(11005, 35)

In [0]:
pred[1]

array([1.9377527e-08, 5.6961790e-07, 1.5162991e-08, 9.9558628e-01,
       2.3715231e-08, 3.1941212e-07, 1.3828520e-07, 4.3812097e-08,
       8.7657446e-09, 6.0813701e-09, 2.5563475e-07, 7.1365054e-07,
       8.3889091e-08, 8.4178770e-05, 3.9931560e-08, 1.1709495e-03,
       1.0500481e-10, 2.1735334e-10, 2.3743554e-07, 9.8820537e-06,
       1.4858299e-10, 2.1984299e-09, 4.7277640e-09, 5.0709804e-07,
       1.4054570e-05, 3.2163469e-05, 6.8258678e-06, 1.0599787e-10,
       3.2139675e-07, 5.9566747e-07, 1.5203410e-07, 6.0299740e-08,
       1.5796627e-09, 3.0914382e-03, 7.6225000e-08], dtype=float32)

In [0]:
labels = np.argmax(pred, axis=-1) 
np.unique(labels, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34]),
 array([141, 258, 185, 204, 200, 435, 409, 446, 165, 122, 421, 411, 192,
        185, 139, 402, 179, 420, 427, 365, 409, 405, 365, 451, 220, 410,
        404, 390, 188, 441, 426, 167, 207, 416, 400]))

In [0]:
words = []
for i in labels:
  if i == 0:
    words.append("backward")
  elif i == 1:
    words.append("bed")
  elif i == 2:
    words.append("bird")
  elif i == 3:
    words.append("cat")
  elif i == 4:
    words.append("dog")
  elif i == 5:
    words.append("down")
  elif i == 6:
    words.append("eight")
  elif i == 7:
    words.append("five")
  elif i == 8:
    words.append("follow")
  elif i == 9:
    words.append("forward")
  elif i == 10:
    words.append("four")
  elif i == 11:
    words.append("go")
  elif i == 12:
    words.append("happy")
  elif i == 13:
    words.append("house")
  elif i == 14:
    words.append("learn")
  elif i == 15:
    words.append("left")
  elif i == 16:
    words.append("marvin")
  elif i == 17:
    words.append("nine")
  elif i == 18:
    words.append("no")
  elif i == 19:
    words.append("off")
  elif i == 20:
    words.append("on")
  elif i == 21:
    words.append("one")
  elif i == 22:
    words.append("right")
  elif i == 23:
    words.append("seven")
  elif i == 24:
    words.append("sheila")
  elif i == 25:
    words.append("six")
  elif i == 26:
    words.append("stop")
  elif i == 27:
    words.append("three")
  elif i == 28:
    words.append("tree")
  elif i == 29:
    words.append("two")
  elif i == 30:
    words.append("up")
  elif i == 31:
    words.append("visual")
  elif i == 32:
    words.append("wow")
  elif i == 33:
    words.append("yes")
  elif i == 34:
    words.append("zero")

In [0]:
print(np.unique(y_train_1))

['backward' 'bed' 'bird' 'cat' 'dog' 'down' 'eight' 'five' 'follow'
 'forward' 'four' 'go' 'happy' 'house' 'learn' 'left' 'marvin' 'nine' 'no'
 'off' 'on' 'one' 'right' 'seven' 'sheila' 'six' 'stop' 'three' 'tree'
 'two' 'up' 'visual' 'wow' 'yes' 'zero']


In [0]:
test['word'] = np.array(words)

,path,word
0,4985e0c3784b700688c35818ba69f01b4fa3e8da.wav,sheila
1,c3815898eb339919ab56249acae83cf566eb622d.wav,yes
2,32c4865f292674cc904c5af503bc669c2dbd8843.wav,bed
3,99600d05d1a861ef9771a7bd8eca0d5f444fce7a.wav,no
4,57ece62e925c94a7e6c54916caca1237467ad4d8.wav,left
...,...,...
11000,21d4a6af950e635bef9d5a4af619ce3cc186206b.wav,go
11001,f8e8f270bbf9c7aeeceb961c5667b622f7baa208.wav,off
11002,223a600994f93fcf869b2731a761b2358dc53c23.wav,five
11003,6e39a17e99a29fa471a063738c821aa53c38c791.wav,left


In [0]:
test['word'] = np.array(words)
test

,path,word
0,4985e0c3784b700688c35818ba69f01b4fa3e8da.wav,six
1,c3815898eb339919ab56249acae83cf566eb622d.wav,cat
2,32c4865f292674cc904c5af503bc669c2dbd8843.wav,seven
3,99600d05d1a861ef9771a7bd8eca0d5f444fce7a.wav,no
4,57ece62e925c94a7e6c54916caca1237467ad4d8.wav,left
...,...,...
11000,21d4a6af950e635bef9d5a4af619ce3cc186206b.wav,go
11001,f8e8f270bbf9c7aeeceb961c5667b622f7baa208.wav,off
11002,223a600994f93fcf869b2731a761b2358dc53c23.wav,five
11003,6e39a17e99a29fa471a063738c821aa53c38c791.wav,left


In [0]:
test

,path,word
0,4985e0c3784b700688c35818ba69f01b4fa3e8da.wav,sheila
1,c3815898eb339919ab56249acae83cf566eb622d.wav,yes
2,32c4865f292674cc904c5af503bc669c2dbd8843.wav,bed
3,99600d05d1a861ef9771a7bd8eca0d5f444fce7a.wav,no
4,57ece62e925c94a7e6c54916caca1237467ad4d8.wav,left
...,...,...
11000,21d4a6af950e635bef9d5a4af619ce3cc186206b.wav,go
11001,f8e8f270bbf9c7aeeceb961c5667b622f7baa208.wav,off
11002,223a600994f93fcf869b2731a761b2358dc53c23.wav,five
11003,6e39a17e99a29fa471a063738c821aa53c38c791.wav,left


In [0]:
test_csv = test.to_csv("/content/drive/test.csv", index = False)